In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from IPython.core.display import display, HTML
from IPython.lib.pretty import pprint
from itertools import chain, tee

In [3]:
from mockdown.model import View

In [4]:
# Gratuitous rainbow bullshit.

HTML("""
<style>
.rainbow-text {
  background-image: repeating-linear-gradient(45deg, violet, indigo, blue, green, yellow, orange, red, violet);
  text-align: center;
  background-size: 800% 800%;
  -webkit-background-clip: text;
  -webkit-text-fill-color: transparent;
  animation: rainbow 4s ease infinite;
}

@keyframes rainbow { 
    0%{background-position:0% 50%}
    50%{background-position:100% 25%}
    100%{background-position:0% 50%}
}
</style>
""")

# Building a View

Always use the `ViewBuilder` class to construct views. This is necessary as we internally keep both `children` and `parent` links, and `View` is a frozen (immutable) dataclass. 

`ViewBuilder` performs some arcane unsafe voodoo to sidestep frozenness and recursively build a doubly-linked tree that `View.__init__` cannot sanely do. 

It also performs more aggressive validation and implements some ✨ <span class="rainbow-text">DSL MAGIC</span> ✨.

In [14]:
from mockdown.model.view import ViewBuilder as V

sample_view = V("p", (0, 0, 100, 100), [
    ("a1", (10, 10, 45, 90), [
        V("b11", (20, 20, 35, 45)),
        V("b12", (20, 55, 35, 80))
    ]),
    ("a2", (55, 10, 90, 90), [
        V("b21", (65, 20, 80, 45)),
        V("b22", (65, 55, 80, 80))
    ])
]).to_view()

# Displaying Views

The `display_view` utility method can be used to display a visual representation of a view hierarchy. 

This method optionally takes `visible_pairs` and `constraints` keyword arguments. If either of these are provided, they will be overlayed. 

In [6]:
from mockdown.display.view import display_view

html_str = str(display_view(sample_view[0], scale=3))
display(HTML(html_str))

In [7]:
from mockdown.visibility import visible_pairs

edge_pairs = visible_pairs(sample_view, deep=True)
anchor_pairs = [(e1.anchor, e2.anchor) for (e1, e2) in edge_pairs]

In [8]:
html_str = str(display_view(sample_view, visible_pairs=edge_pairs, scale=3))
display(HTML(html_str))

In [9]:
from mockdown.logic import valid_constraints
constraints = list(valid_constraints(sample_view, anchor_pairs))

for constraint in constraints[0:10]:
    print(str(constraint))
print("...")

a2.left ≤ a1.right (priority=1000, samples=0, kind=spacing)
a2.left ≥ a1.right (priority=1000, samples=0, kind=spacing)
b12.top ≤ b11.bottom (priority=1000, samples=0, kind=spacing)
b12.top ≥ b11.bottom (priority=1000, samples=0, kind=spacing)
b22.top ≤ b21.bottom (priority=1000, samples=0, kind=spacing)
b22.top ≥ b21.bottom (priority=1000, samples=0, kind=spacing)
a1.top ≤ p.top (priority=1000, samples=0, kind=spacing)
a1.top ≥ p.top (priority=1000, samples=0, kind=spacing)
a1.left ≤ p.left (priority=1000, samples=0, kind=spacing)
a1.left ≥ p.left (priority=1000, samples=0, kind=spacing)
...


In [10]:
html_str = str(display_view(sample_view, constraints=constraints, scale=3))
display(HTML(html_str))

In [11]:
trained_constraints = [constraint.train_view(sample_view) for constraint in constraints]
    
for constraint in trained_constraints:
    print(str(constraint))
print("...")

a2.left ≤ a1.right + 10 (priority=1000, samples=1, kind=spacing)
a2.left ≥ a1.right + 10 (priority=1000, samples=1, kind=spacing)
b12.top ≤ b11.bottom + 10 (priority=1000, samples=1, kind=spacing)
b12.top ≥ b11.bottom + 10 (priority=1000, samples=1, kind=spacing)
b22.top ≤ b21.bottom + 10 (priority=1000, samples=1, kind=spacing)
b22.top ≥ b21.bottom + 10 (priority=1000, samples=1, kind=spacing)
a1.top ≤ p.top + 10 (priority=1000, samples=1, kind=spacing)
a1.top ≥ p.top + 10 (priority=1000, samples=1, kind=spacing)
a1.left ≤ p.left + 10 (priority=1000, samples=1, kind=spacing)
a1.left ≥ p.left + 10 (priority=1000, samples=1, kind=spacing)
a1.bottom ≤ p.bottom - 10 (priority=1000, samples=1, kind=spacing)
a1.bottom ≥ p.bottom - 10 (priority=1000, samples=1, kind=spacing)
a2.top ≤ p.top + 10 (priority=1000, samples=1, kind=spacing)
a2.top ≥ p.top + 10 (priority=1000, samples=1, kind=spacing)
a2.bottom ≤ p.bottom - 10 (priority=1000, samples=1, kind=spacing)
a2.bottom ≥ p.bottom - 10 (prio